# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234432708341                   -0.50    8.0
  2   -7.250330294518       -1.80       -1.41    1.0
  3   -7.251168224425       -3.08       -1.97    4.0
  4   -7.250939844828   +   -3.64       -1.87    4.0
  5   -7.251336715968       -3.40       -2.95    3.0
  6   -7.251338338980       -5.79       -3.24    3.0
  7   -7.251338790054       -6.35       -4.13    2.0
  8   -7.251338796679       -8.18       -4.37    4.0
  9   -7.251338798598       -8.72       -4.86    3.0
 10   -7.251338798633      -10.45       -5.03    1.0
 11   -7.251338798703      -10.16       -5.97    3.0
 12   -7.251338798704      -11.77       -6.46    3.0
 13   -7.251338798705      -13.16       -6.83    2.0
 14   -7.251338798705      -14.27       -7.33    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.047205602577258636
[ Info: Arnoldi iteration step 2: normres = 0.6140561276935238
[ Info: Arnoldi iteration step 3: normres = 0.8424301264702035
[ Info: Arnoldi iteration step 4: normres = 0.21401493912429137
[ Info: Arnoldi iteration step 5: normres = 0.6206389259486044
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.75e-02, 8.60e-02, 5.83e-01, 1.93e-01, 5.82e-03)
[ Info: Arnoldi iteration step 6: normres = 0.18304646681448217
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.07e-03, 1.06e-01, 9.67e-02, 8.21e-02, 2.17e-02)
[ Info: Arnoldi iteration step 7: normres = 0.13227177854624608
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.60e-04, 1.24e-02, 1.15e-02, 1.09e-01, 6.61e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11560673459904641
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.32